In [18]:
%matplotlib inline

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

# Data Wrangling

All data on cities and counties were sourced from https://www.census.gov/quickfacts.

Because we will be analyzing the impact, if any, of variously ranked colleges on the respective cities and counties they are located in, we want the data to be organized based on ranking, allowing us to easily isolate the analysis.

The U.S. Census Bureau provides us tables with the following data.

In [19]:
cities_dataset_path = './datasets/cities/'
counties_dataset_path = './datasets/counties/'

# create separate dictionaries for each category of colleges that the cities are home to
top_cities = dict()
mid_cities = dict()
bot_cities = dict()
other_cities = dict() # misc areas for further analysis

# populate dicts based on filename indicators
for c in os.listdir(cities_dataset_path):
    # filenames truncated to city-state format for simplicity
    truncated_file_name = '-'.join((c.split('-')[:2]))
    if c.endswith("top-city.csv"):
        top_cities[truncated_file_name] = pd.read_csv(cities_dataset_path + c)
    elif c.endswith("mid-city.csv"):
        mid_cities[truncated_file_name] = pd.read_csv(cities_dataset_path + c)
    elif c.endswith("bot-city.csv"):
        bot_cities[truncated_file_name] = pd.read_csv(cities_dataset_path + c)
    elif '-' in c:
        other_cities[truncated_file_name] = pd.read_csv(cities_dataset_path + c)

# similar process for counties
top_counties = dict()
mid_counties = dict()
bot_counties = dict()
other_counties = dict()

for c in os.listdir(counties_dataset_path):
    # filenames truncated to county-state format for simplicity
    truncated_file_name = '-'.join((c.split('-')[:2]))
    if c.endswith("top-county.csv"):
        top_counties[truncated_file_name] = pd.read_csv(counties_dataset_path + c)
    elif c.endswith("mid-county.csv"):
        mid_counties[truncated_file_name] = pd.read_csv(counties_dataset_path + c)
    elif c.endswith("bot-county.csv"):
        bot_counties[truncated_file_name] = pd.read_csv(counties_dataset_path + c)
    elif '-' in c:
        other_counties[truncated_file_name] = pd.read_csv(counties_dataset_path + c)

# testing
print('============== TOP TEST ===============')
print(top_cities['stanford-ca'])
print('\n============== MID TEST ===============')
print(mid_cities['whitewater-wi'])
print('\n============== BOT TEST ===============')
print(bot_counties['westmoreland-pa'])

============== TOP TEST ===============
                                                 Fact  \
0        Population estimates, July 1, 2018,  (V2018)   
1   Population estimates base, April 1, 2010,  (V2...   
2   Population, percent change - April 1, 2010 (es...   
3                   Population, Census, April 1, 2010   
4                      Persons under 5 years, percent   
5                     Persons under 18 years, percent   
6                  Persons 65 years and over, percent   
7                             Female persons, percent   
8                                White alone, percent   
9            Black or African American alone, percent   
10   American Indian and Alaska Native alone, percent   
11                               Asian alone, percent   
12  Native Hawaiian and Other Pacific Islander alo...   
13                         Two or More Races, percent   
14                        Hispanic or Latino, percent   
15       White alone, not Hispanic or Latino, pe

# Data Cleaning

The first part of the data we wanted to clean was simply the name of the column that holds all the statistic values. The name was originally just the name of the county/city. We believe that changing it to "fact_value" (given that the column with the name of the statistic is "fact", would be more clear.

In [20]:
all_csv = [top_cities, mid_cities, bot_cities, other_cities, top_counties, mid_counties, bot_counties, other_counties]
for d in all_csv:
    for k, v in d.items():
        v.rename(index = str, columns = {v.columns[2] : 'Fact Value'}, inplace = True)

assert top_cities['cambridge-ma'].columns[2] == 'Fact Value'

Currently, the column that should hold the fact value is of type object (str). We want this to be converted to ints so that we do not need to constantly type cast when performing our analysis.

To do this, we need to first delete rows that a fact value that indicates this statistic is not useful such as NA for not available. Further descriptions of the invalid value strings are given in a following section.

In [21]:
assert len(bot_cities['redding-ca'].loc[bot_cities['redding-ca']['Fact Value'] == 'X']) != 0
invalid_values = ['X', 'NA', 'D', '-', 'FN', 'F', 'S', 'Z']
for d in all_csv:
    for k, v in d.items():
        for i, r in v.iterrows():
            if r['Fact Value'] in invalid_values:
                v.drop(i, inplace = True)
                

assert len(bot_cities['redding-ca'].loc[bot_cities['redding-ca']['Fact Value'] == 'X']) == 0

Now, we can iterate through the Fact Value column, strip all non-numeric or . (indicating a decimal number) characters and retype each value from str to float64. We print out the pre and post cleaning types of the Fact Value column to verify our code.

In [22]:
print('Before cleaning:')
print(mid_counties['walworth-wi']['Fact Value'].dtypes)

for d in all_csv:
    for k, v in d.items():
        v.loc[:, 'Fact Value'] = v.loc[:, 'Fact Value'].str.strip('%"$')
        v.loc[:, 'Fact Value'] = v.loc[:, 'Fact Value'].str.replace(',', '')   
        v.loc[:, 'Fact Value'] = v.loc[:, 'Fact Value'].astype('float64')
        
print('\nAfter cleaning:')
print(mid_counties['walworth-wi']['Fact Value'].dtypes)

Before cleaning:
object

After cleaning:
float64


As seen in the initial output of our data wrangling, there are a couple of entirely or largely unneeded *columns* in the data where all the values are either NaN or irrelevant. 

We will be dropping or modifying the following columns to clean the data:

###### Fact Note:

This indicates:

**(a)** Includes persons reporting only one race

**(b)** Hispanics may be of any race, so also are included in applicable race categories

**(c)** Economic Census - Puerto Rico data are not comparable to U.S. Economic Census Data

As none of these factors influence the demographic data of the areas we are analyzing, we can remove this data.

###### Value Note 

This indicates: 

**(-)** Either no or too few sample observations were available to compute an estimate, or a ratio of medians cannot be calculated because one or both of the median estimates falls in the lowest or upper interval of an open ended distribution.	

**(D)**	Suppressed to avoid disclosure of confidential information	

**(F)**	Fewer than 25 firms		

**(FN)** Footnote on this item in place of data	

**(NA)** Not available		

**(S)**	Suppressed; does not meet publication standards

**(X)**	Not applicable

**(Z)**	Value greater than zero but less than half unit of measure shown

Value notes are indicated in the Fact Value column so we do not need the Value Note column at all.

In [23]:
assert top_cities['stanford-ca'].shape == (67, 4)

for d in all_csv:
    for k, v in d.items():
        v.drop([v.columns[1], v.columns[3]], axis = 1, inplace = True)

assert top_cities['stanford-ca'].shape == (67, 2)

We also observe that there are also many *rows* that have a NaN value.

In [24]:
for d in all_csv:
    for k, v in d.items():
        print('NaN values in each column for ' + k + ':\n'+ str(v.isna().sum()))

NaN values in each column for cambridge-ma:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for newhaven-ct:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for stanford-ca:
Fact           5
Fact Value    21
dtype: int64
NaN values in each column for jacksonville-il:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for westerville-oh:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for whitewater-wi:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for greensburg-pa:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for redding-ca:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for sanbernardino-ca:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for berkeley-ca:
Fact           5
Fact Value    23
dtype: int64
NaN values in each column for middlesex-ma:
Fact           5
Fact Value    20
dtype:

These NaN rows are not useful, so we will drop these rows entirely.

In [25]:
for d in all_csv:
    for k, v in d.items():
        v.dropna(inplace = True)
        print('NaN values in each column for ' + k + ':\n'+ str(v.isna().sum()))

NaN values in each column for cambridge-ma:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for newhaven-ct:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for stanford-ca:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for jacksonville-il:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for westerville-oh:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for whitewater-wi:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for greensburg-pa:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for redding-ca:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for sanbernardino-ca:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for berkeley-ca:
Fact          0
Fact Value    0
dtype: int64
NaN values in each column for middlesex-ma:
Fact          0
Fact Value    0
dtype: int64
NaN values in e

Because our analysis will only need statistics about demographics, we can drop all rows with irrelevant statistics. We decided to do this last so that, in case we do need to use other statistics, the data for that row will already be cleaned up to this point.

In [26]:
demographic_rows = ['White alone, percent', 'Black or African American alone, percent', 'American Indian and Alaska Native alone, percent', 'Asian alone, percent', 'Native Hawaiian and Other Pacific Islander alone, percent', 'Two or More Races, percent', 'Hispanic or Latino, percent', 'White alone, not Hispanic or Latino, percent']

for d in all_csv:
    for k, v in d.items():
        d[k] = v.loc[(v['Fact'].isin(demographic_rows))]
        # reset index to start from 0 since rows before may have been dropped
        d[k].reset_index(drop = True, inplace = True)

for d in all_csv:
    for k, v in d.items():
        assert d[k].shape == (7, 2) or d[k].shape == (8, 2)

We are now left with dataframes that are of shape either (7, 2) or (8, 2), have no NaN values, have values that are ready to analyze and work with (float type) and only contain relevant demographic statistics.

#### Here is the final cleaned data:

In [27]:
for d in all_csv:
    for k, v in d.items():
        print(k + ' | shape: ' + str(v.shape))
        print(v)
        print('\n')

cambridge-ma | shape: (8, 2)
                                                Fact  Fact Value
0                               White alone, percent        66.9
1           Black or African American alone, percent        10.8
2   American Indian and Alaska Native alone, percent         0.2
3                               Asian alone, percent        15.7
4  Native Hawaiian and Other Pacific Islander alo...         0.1
5                         Two or More Races, percent         4.2
6                        Hispanic or Latino, percent         8.8
7       White alone, not Hispanic or Latino, percent        61.6


newhaven-ct | shape: (8, 2)
                                                Fact  Fact Value
0                               White alone, percent        43.0
1           Black or African American alone, percent        33.0
2   American Indian and Alaska Native alone, percent         0.3
3                               Asian alone, percent         4.7
4  Native Hawaiian and Other Pa

In [28]:
#convert all_csv to dataframe
columns = demographic_rows
df = pd.DataFrame()

i1 = 0;
for a in all_csv:
    index = 0
    for b in a.keys():
        i2 = 0
        for x in range(0, 8):
            if x < len(all_csv[i1].get(b)["Fact Value"]):
                df.loc[x,b] = (all_csv[i1].get(b)["Fact Value"][i2])
            i2 = i2 + 1
    i1 = i1 + 1
    index = index + 1
df_cities = df.transpose()
df_cities

,0,1,2,3,4,5,6,7
cambridge-ma,66.9,10.8,0.2,15.7,0.1,4.2,8.8,61.6
newhaven-ct,77.9,14.7,0.5,4.3,0.1,2.4,18.1,63.0
stanford-ca,56.6,4.3,0.4,27.0,0.3,9.7,13.6,47.4
jacksonville-il,84.9,11.0,0.3,1.0,0.0,2.6,3.9,81.8
westerville-oh,86.2,8.0,0.0,2.1,0.0,3.2,1.9,84.6
whitewater-wi,87.9,3.8,0.1,2.1,0.6,2.9,9.6,81.8
greensburg-pa,88.0,6.1,0.0,1.4,0.0,4.4,2.1,86.3
redding-ca,84.3,1.4,2.2,4.8,0.0,4.7,10.1,77.9
sanbernardino-ca,76.9,9.4,2.1,7.6,0.5,3.5,53.4,28.6
berkeley-ca,60.2,8.6,0.5,19.7,0.5,7.1,11.0,54.6
